In [16]:
import numpy as np
import pandas as pd
from scipy.stats import norm

In [17]:
df = pd.read_excel(
        'Data/VaR y VaRajustado por liquidez.xlsx',
        header=2
    )

df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')

**a) Calcular el VaR en porcentaje y en dinero para cada día de enero de 2026 para una posición de 128 mil contratos de futuro de plata (long, hedge fund) con método histórico nivel de confianza 97.5%**

In [18]:
pa = 128000  # contratos de futuro de plata

returns = df['Settlement Price'].pct_change().dropna()

var_pct = returns.rolling(window=250).apply(lambda x: -np.percentile(x, 2.5))

last_price = 5000  # precio de la plata en el último día

enero = df['Date'].dt.year.eq(2026) & df['Date'].dt.month.eq(1)

var_money = var_pct * last_price * pa * df['Settlement Price']

var_a = pd.DataFrame({
    'Date': df.loc[enero, 'Date'],
    'VaR_%': var_pct.loc[enero] * 100,
    'VaR_$': var_money.loc[enero]
}).dropna()

var_a

,Date,VaR_%,VaR_$
0,2026-01-30,6.897934,3.466891e+09
1,2026-01-29,5.692781,4.169083e+09
2,2026-01-28,5.692781,4.136475e+09
3,2026-01-27,5.692781,3.860416e+09
4,2026-01-26,4.571859,3.379636e+09
5,2026-01-23,4.571859,2.964993e+09
6,2026-01-22,4.571859,2.819835e+09
7,2026-01-21,4.571859,2.710549e+09
8,2026-01-20,4.571859,2.769040e+09
9,2026-01-16,4.571859,2.590584e+09


**b) Calcular el VaR en porcentaje y en dinero para cada día de enero de 2026 paa una posición de -239 mil contratos (short, minera), método histórico nivel de confianza 99%**

In [19]:
pb = 239000  # contratos de futuro de plata

var_pct = returns.rolling(window=251).apply(lambda x: np.percentile(x, 99))

var_money = var_pct * last_price * pb * df['Settlement Price']

var_b = pd.DataFrame({
    'Date': df.loc[enero, 'Date'],
    'VaR_%': var_pct.loc[enero] * 100,
    'VaR_$': var_money.loc[enero]
}).dropna()

var_b

,Date,VaR_%,VaR_$
0,2026-01-30,7.816301,7.335172e+09
1,2026-01-29,7.816301,1.068822e+10
2,2026-01-28,7.816301,1.060462e+10
3,2026-01-27,7.816301,9.896892e+09
4,2026-01-26,7.816301,1.078863e+10
5,2026-01-23,7.467500,9.042615e+09
6,2026-01-22,7.467500,8.599912e+09
7,2026-01-21,7.467500,8.266613e+09
8,2026-01-20,7.467500,8.444997e+09
9,2026-01-16,7.467500,7.900743e+09


**c) Calcular el VaR ajustado por liquidez solo para el lunes 2-feb-2026 para la posición long de 128 mil contratos, mismo método y nivel de confianza que inciso "a"**

In [20]:
df['spread_pct'] = (df['Ask'] - df['Bid']) / ((df['Ask'] + df['Bid']) / 2)
mean_spread = df['spread_pct'].tail(251).mean()
std_spread = df['spread_pct'].tail(251).std()

In [21]:
z_a = norm.ppf((1 + 0.975) / 2) # valor z para 97.5% de confianza

lvar = var_a.iloc[0, 1]/100 + 0.5 * (mean_spread + z_a * std_spread)
lvar * 100

np.float64(7.157494366646576)

]**d) Calcular el VaR ajustado por liquidez solo para el lunes 2-feb-2026 para la posición short de 239 mil contratos, mismo método y nivel de confianza que inciso "b"**

In [22]:
z_b = norm.ppf((1 + 0.99) / 2) # valor z para 99% de confianza

lvar = var_b.iloc[0, 1]/100 + 0.5 * (mean_spread + z_b * std_spread)
lvar * 100

np.float64(8.105376264788301)

**¿Cuántos días tomaría deshacer la posición larga de 128 mil futuros siguiendo un lineamiento o política de máximo negociar el 10% del volumen promedio últimos 90 días?**

In [23]:
mean_volume = np.floor((df['CVol'].tail(90).mean())*0.1)

ndays = np.round(pa / mean_volume,2)
ndays

np.float64(10.72)

**¿Si la política de liquidez en trading para la minera es que ninguna posición tome más de 10 días hábiles en deshacerse limitandose a negociar hasta el 20% del volumen promedio últimos 90 días, cumple o no cumple?**

In [24]:
mean_volume = np.floor((df['CVol'].tail(90).mean())*0.2)

ndays_a = np.round(pa / mean_volume,2)

ndays_b = np.round(pb / mean_volume,2)

ndays_a, ndays_b

(np.float64(5.36), np.float64(10.01))

**Respuestas:** No cumple